# PARTE 02


## Data import
Qui vengon importati i dati delle azioni (prince out) in formato dataframe in pandas.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [6]:
# Leggi indici filtrati per ottenere l'indice di mercato 
sp5 = pd.read_csv('Data_filtrati_indici.csv')

# togli ^ dalla colonna
sp5.columns = [col.replace('^', '') for col in sp5.columns]

sp5 = sp5.rename(columns={'SPX': 'SP500'})

# Leggi il file di altre stokes
df = pd.read_csv('Dati_non_filtrati_200.csv')

#  Aggiungi la colonna SP500 (il df_sp che hai preparato prima)
df.insert(1, 'SP500', sp5['SP500'].values)

# tolgo la prima colonna Date
df = df.drop(columns=['Date'])

print(df.head())

print('\nil dataframe è grande:')
print(df.shape)

         SP500       ABCB      AACG          A        ACN         AD  \
0  2058.199951  22.140774  0.450767  37.120949  74.252625  27.755260   
1  2020.579956  22.123255  0.423913  36.425381  72.998894  24.875130   
2  2002.609985  21.667646  0.432545  35.857952  72.472366  24.686842   
3  2025.900024  21.457361  0.417199  36.333858  73.993515  25.195921   
4  2062.139893  21.641356  0.405690  37.422966  75.121864  24.910000   

         ABG   ACAD      ABEV        ACM  ...   CALX        BXP       CBRL  \
0  76.089996  31.15  3.944752  29.278265  ...  10.01  86.060890  85.954865   
1  74.000000  31.10  3.871825  27.995228  ...   9.82  86.841118  85.124863   
2  72.690002  30.98  4.017682  27.715469  ...   9.73  88.526138  82.994125   
3  75.010002  32.16  4.083980  28.284634  ...   9.66  89.516197  84.610764   
4  75.500000  32.50  4.127321  29.162498  ...   9.73  89.876778  84.375381   

        CASY        CBIO   BXC        CCL        BZH       CBOE        BYD  
0  82.087761 -818.400

## Calcolo ritorno e covarianza delle singole azioni
Inizio a calcolare i ritoni gironalieri, la matrice covarianza rolling per ogni giorno a blocchi di 30 giori( rolling variance vuol dire che calcolo la cov in un intervallo W qui 30 )

In [ ]:
#calcolo i ritorni e la covarianza mobile 

returns = df.pct_change().dropna()  #  pct_change calcola i ritorni percentuali e dropna rimuove la prima riga NaN
print("--- Nomi degli Indici (Tempi) ---")
print(returns.index)
print("\nRitorni:")
print(returns.head())
print(returns.shape)

W = 30 # Finestra di 30 giorni
print(f"Calcolo della Matrice di Covarianza Mobile (Rolling) con W={W}")


# L'output è un oggetto Series con MultiIndex (data, asset), 
# ma rappresenta una matrice 3x3 per ogni giorno 't'
rolling_cov_series = returns.rolling(window=W).cov()

# Rimuovi i NaN iniziali (le prime W-1 righe)
rolling_cov_series = rolling_cov_series.dropna()
print("\nMatrice di Covarianza Mobile (Rolling Covariance Matrices):")
print(rolling_cov_series)

# Il MultiIndex (o Indice Multi-Livello) è la struttura di dati che Pandas utilizza per memorizzare in modo efficiente le matrici
# 3 x 3(un oggetto 3D) all'interno di un oggetto 2D (Series o DataFrame).La dimensione dell'output è $(153, 3)$.
# Questo significa che l'oggetto è una tabella con 153 righe e 3 colonne -- OVVERO 153/3 SONO INFATTI 51 MATRICI 3X3

print(f"\nDimensione totale (MultiIndex): {rolling_cov_series.shape}")
print(f"Asset totali: {len(returns.columns)}")
print(f"Matrici di Covarianza generate: {len(rolling_cov_series) / len(returns.columns)}")



--- Nomi degli Indici (Tempi) ---
Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       2506, 2507, 2508, 2509, 2510, 2511, 2512, 2513, 2514, 2515],
      dtype='int64', length=2515)
      SP500      ABCB      AACG         A       ACN        AD       ABG  \
1 -0.018278 -0.000791 -0.059574 -0.018738 -0.016885 -0.103769 -0.027467   
2 -0.008893 -0.020594  0.020362 -0.015578 -0.007213 -0.007569 -0.017703   
3  0.011630 -0.009705 -0.035477  0.013272  0.020989  0.020621  0.031916   
4  0.017888  0.008575 -0.027586  0.029975  0.015249 -0.011348  0.006532   
5 -0.008404 -0.029555  0.014184 -0.007337 -0.001113 -0.006159 -0.013907   

       ACAD      ABEV       ACM  ...      CALX       BXP      CBRL      CASY  \
1 -0.001605 -0.018487 -0.043822  ... -0.018981  0.009066 -0.009656 -0.021910   
2 -0.003859  0.037671 -0.009993  ... -0.009165  0.019403 -0.025031 -0.007352   
3  0.038089  0.016502  0.020536  ... -0.007194  0.011184  0.019479  0.011457   
4  0.010572  0.

In [ ]:
#controllo che la dimensione della matrice fatta di matrici covarianza sia giusta (sarebbero 199 in lunghezza per ogni giorno (2515-29) [perchè la covarainza è calxolata ogni 30 gg quidni tolgo 29 ] -> 494714)
rolling_cov_series.shape

(494714, 199)

## Maschera per ottenere pesi casuali per il portafoglio.
Creare una matrice saprsa da N colonne(numero totale delle  stokes) in cui solo 100 valori per ogni riga siano diversi da zero e la somma faccia uno. Ogni riga equivale ai pesi per le 100 stokes usate per il portafoglio sommate attraverso i pesi.

In [10]:
#creo matrice di pesi casuali per il portafoglio
n_stocks=df.shape[1]    #qui va messo il numero di stocks scaricate quindi le 
num_portfolios = 20    # numero portafogli da creare casuali
n_selected_stocks = 100  # numero Stock da usare nel portafoglio

# --- Inizializzazione ---
portfolio_weights = np.zeros((num_portfolios, n_stocks))

# --- Loop di Generazione ---
for i in range(num_portfolios):
    # 1. Seleziono ESATTAMENTE 100 indici casuali senza ripetizione
    #    -> size è impostato a n_selected_stocks (100)
    selected_indices = np.random.choice(n_stocks, size=n_selected_stocks, replace=False)
    
    # 2. Genero ESATTAMENTE 100 pesi casuali
    #    -> L'argomento è impostato a n_selected_stocks (100)
    random_weights = np.random.rand(n_selected_stocks)
    
    # 3. Normalizzo i pesi in modo che sommino a 1
    normalized_weights = random_weights / random_weights.sum()
    
    # 4. Assegno i pesi normalizzati solo alle 100 stock selezionate.
    #    Le restanti (n_stocks - 100) rimangono a 0 grazie a np.zeros() iniziale.
    portfolio_weights[i, selected_indices] = normalized_weights

print("Matrice di pesi casuali per i portafogli:")
print(portfolio_weights)
print("\nVerifica della matrice:")
print(f"Shape: {portfolio_weights.shape}")


# Verifica che i pesi di ogni riga sommino a 1
sums = portfolio_weights.sum(axis=1)
print(f"Somma dei pesi del primo portafoglio: {sums[0]:.6f}")

# Verifica che ogni portafoglio abbia esattamente 100 pesi > 0
nonzero_counts = np.count_nonzero(portfolio_weights, axis=1)
print(f"Numero di stock con peso > 0 per i primi 5 portafogli: {nonzero_counts[:5]}")

#stampa la prima riga della matrice dei pesi
print("Pesi del primo portafoglio:")
print(portfolio_weights[0])

Matrice di pesi casuali per i portafogli:
[[0.         0.01543525 0.01424205 ... 0.01611023 0.00184771 0.        ]
 [0.01715607 0.01280052 0.01487058 ... 0.         0.         0.        ]
 [0.00891001 0.01157939 0.         ... 0.00271614 0.01617948 0.00052253]
 ...
 [0.         0.01374598 0.01080078 ... 0.02074677 0.00582906 0.01178624]
 [0.00874955 0.         0.         ... 0.         0.         0.        ]
 [0.00580154 0.         0.00451028 ... 0.01509755 0.0001334  0.        ]]

Verifica della matrice:
Shape: (20, 199)
Somma dei pesi del primo portafoglio: 1.000000
Numero di stock con peso > 0 per i primi 5 portafogli: [100 100 100 100 100]
Pesi del primo portafoglio:
[0.00000000e+00 1.54352492e-02 1.42420531e-02 4.06757880e-03
 1.14188902e-02 0.00000000e+00 1.16414276e-02 0.00000000e+00
 4.82214365e-03 0.00000000e+00 0.00000000e+00 7.22264065e-03
 4.06081007e-03 3.62162014e-03 8.13750008e-03 0.00000000e+00
 7.58623699e-03 0.00000000e+00 2.07861931e-03 9.85044389e-03
 0.00000000e+00

## Calcolo le feauture del portafoglio (prezzi, return, var, beta)
Calcolo i prezzi e ritorni del portafoglio come somma pesata dei prezzi e ritorni delle azioni, la varianza giornaliera come radice di (pesi trasposti* matrice cov*pesi * 252), e la beta come 

In [14]:
# calcolo ritorni, prezzi , volatilità del portafoglio

#creo una matrice dove salavo la volatilità di ogni portafoglio
portfolio_volatility_matrix = pd.DataFrame(index=rolling_cov_series.index.get_level_values(0).unique()) #prende le date (primo livello del MultiIndex) e costruisce un Index delle date valide:
portfolio_out = pd.DataFrame(index=df.index)
portfolio_returns = pd.DataFrame(index=returns.index)

#faccio un ciclo per ogni portafoglio con i suoi pesi presi dalla maschera
for p in range(num_portfolios):
    weights = portfolio_weights[p]
    portfolio_returns[f"Portafoglio {p+1}"] = returns.dot(weights)
    portfolio_out[f"Portafoglio {p+1}"] = df.dot(weights)


    #calcolo volatilità del portafoglio nel tempo usando la matrice di covarianza mobile

    # Dizionario per salvare la serie temporale della volatilità
    portfolio_volatility_mobile = {}

    # Itera su ogni data (t) nel risultato mobile valido
    for date in rolling_cov_series.index.get_level_values(0).unique():
        
        # Estrai la Matrice di Covarianza (Σt) per la data corrente
        # .values converte il DataFrame 3x3 in un array NumPy
        cov_matrix_t = rolling_cov_series.loc[date].values
        
        # Calcolo della Varianza del Portafoglio: w.T * Σt * w
        portfolio_variance_t = weights.T.dot(cov_matrix_t).dot(weights)
        
        # Volatilità annualizzata al tempo t
        # np.sqrt(...) * np.sqrt(252)
        portfolio_volatility_annual_t = np.sqrt(portfolio_variance_t) * np.sqrt(252)
        
        # Salva il risultato nel dizionario
        portfolio_volatility_mobile[date] = portfolio_volatility_annual_t

    # ----------------------------------------------------
    # 4. RISULTATO FINALE
    # ----------------------------------------------------

    # Converte il risultato in una Serie Pandas 
    portfolio_volatility_series = pd.Series(portfolio_volatility_mobile)

   #aggiungo una colonna di colatilità per ogni azione
    portfolio_volatility_matrix[f"Portafoglio {p+1}"] = portfolio_volatility_series

print("\nRitorni dei portafogli nel tempo:")
print(portfolio_returns.shape)
print(portfolio_returns)

print("\nPrezzi dei portafogli nel tempo:")
print(portfolio_out)

print("\nVolatilità dei portafogli nel tempo:")
print(portfolio_volatility_matrix.shape)
portfolio_volatility_matrix.head()




Ritorni dei portafogli nel tempo:
(2515, 20)
      Portafoglio 1  Portafoglio 2  Portafoglio 3  Portafoglio 4  \
1         -0.010883      -0.013174      -0.010059      -0.007301   
2         -0.007720      -0.008038      -0.011933      -0.007154   
3          0.011854       0.014538       0.013711       0.010569   
4          0.013118       0.008608       0.009376       0.010766   
5          0.005645      -0.001140      -0.003377      -0.002239   
...             ...            ...            ...            ...   
2511       0.003822       0.003702       0.008231       0.005514   
2512       0.007134       0.007086       0.005931       0.008429   
2513      -0.009140      -0.008900      -0.007929      -0.009328   
2514      -0.002581      -0.002827      -0.006344      -0.007701   
2515       0.002074       0.003789       0.003308       0.000047   

      Portafoglio 5  Portafoglio 6  Portafoglio 7  Portafoglio 8  \
1         -0.010295      -0.009947      -0.009354      -0.012472   
2

,Portafoglio 1,Portafoglio 2,Portafoglio 3,Portafoglio 4,Portafoglio 5,Portafoglio 6,Portafoglio 7,Portafoglio 8,Portafoglio 9,Portafoglio 10,Portafoglio 11,Portafoglio 12,Portafoglio 13,Portafoglio 14,Portafoglio 15,Portafoglio 16,Portafoglio 17,Portafoglio 18,Portafoglio 19,Portafoglio 20
30,0.152615,0.138493,0.118917,0.113921,0.125294,0.128628,0.145857,0.128596,0.118595,0.108025,0.129038,0.114413,0.142331,0.117710,0.110895,0.146318,0.126908,0.154468,0.140230,0.117601
31,0.148774,0.132046,0.115925,0.111644,0.120423,0.124172,0.142283,0.121865,0.114698,0.105005,0.126273,0.111885,0.135783,0.110644,0.109226,0.140514,0.122948,0.150687,0.137722,0.115300
32,0.146090,0.129023,0.109163,0.108454,0.116667,0.120477,0.139045,0.119288,0.111982,0.101786,0.123257,0.106349,0.130561,0.104632,0.106112,0.136334,0.117300,0.145470,0.134283,0.112556
33,0.143219,0.123776,0.103266,0.105704,0.108798,0.118469,0.137420,0.117606,0.107954,0.098252,0.121446,0.106457,0.126059,0.102618,0.095484,0.133703,0.111097,0.141294,0.129437,0.106391
34,0.139147,0.121798,0.100376,0.102622,0.106448,0.116225,0.132471,0.112837,0.105619,0.095897,0.118240,0.105728,0.125941,0.101288,0.092767,0.132606,0.108861,0.140544,0.126157,0.104168


In [13]:
#calcolo beta rispetto a sp500 di ogni portafoglio
# returns deve essere il tuo DataFrame di ritorni decimali (Tempi x Asset)
sp500_returns = returns['SP500']
beta_mobile_df = pd.DataFrame(index=returns.index)

# 1. Calcola la Varianza Mobile dell'Indice di Mercato (nella formula di beta il denominatore)
# Questa serie temporale contiene la varianza dell'S&P 500 al tempo t.
rolling_variance_sp500 = sp500_returns.rolling(window=W).var()  #uso la solita funzione rolling con finestra W
rolling_variance_sp500 = rolling_variance_sp500.dropna()

# 2. Itera su tutti gli asset (colonne) per calcolare il Beta mobile per ciascuno
for stock in portfolio_returns.columns:
    if stock != 'SP500':
        
        # 3. Calcola la Covarianza Mobile dell'Asset con l'S&P 500 (numeratore)
        # La funzione .cov() è applicata in modo rolling tra le due Series.
        rolling_covariance = portfolio_returns[stock].rolling(window=W).cov(sp500_returns)
        
        # 4. Calcola il Beta Mobile: Covarianza Mobile / Varianza Mobile
        # Questa operazione avviene in modo "vettoriale" su tutta la serie temporale.
        beta_t = rolling_covariance / rolling_variance_sp500
        
        # 5. Salva la serie temporale del Beta nell'output finale
        beta_mobile_df[stock] = beta_t

# Rimuovi le prime W-1 righe (NaN) che non avevano abbastanza dati
beta_mobile_df = beta_mobile_df.dropna()

print("Matrice di Beta Mobile (Serie Temporale) rispetto sp500:")
print(f"Dimensione (Giorni x Asset): {beta_mobile_df.shape}")
print(beta_mobile_df.head())



print("\nOut portfolio :")
print(portfolio_out.shape)
print(portfolio_out.head())

print("\nRitorni portfolio:")
print(portfolio_returns.shape)
print(portfolio_returns.head())


Matrice di Beta Mobile (Serie Temporale) rispetto sp500:
Dimensione (Giorni x Asset): (2486, 20)
    Portafoglio 1  Portafoglio 2  Portafoglio 3  Portafoglio 4  Portafoglio 5  \
30       0.751738       0.715610       0.644776       0.585408       0.666348   
31       0.765721       0.710300       0.652700       0.601603       0.670772   
32       0.757081       0.697549       0.624869       0.588064       0.656737   
33       0.735515       0.658279       0.587435       0.565344       0.612273   
34       0.738387       0.685960       0.598775       0.568241       0.632360   

    Portafoglio 6  Portafoglio 7  Portafoglio 8  Portafoglio 9  \
30       0.686651       0.768665       0.717779       0.647186   
31       0.694831       0.794782       0.715905       0.655762   
32       0.681236       0.783356       0.710261       0.644610   
33       0.666652       0.776238       0.692683       0.617347   
34       0.691794       0.780845       0.695307       0.637382   

    Portafoglio 10 

## Creo matrice di imput della rete neurale (freauture)
Creo un tensore in 2 dimensioni:
<br>1 dim le 4 feature calcolate(prezzo, return, volatilità, beta) per ogni asset del portafoglio
<br>2 dim dei 2486 giorni
<br>(non ho fatto la terza perchè avevamo cambiato più volte la decisione su come fare l'imput la posso fare domani se vuoi)

In [15]:
feature_matrix = {}

for p in range(num_portfolios):
    portfolio_name = f"Portafoglio {p+1}"
    suffix = f"_P{p+1}"

    # Crea la matrice delle feature
    feature_matrix[portfolio_name] = pd.DataFrame({
        f'Prezzo{suffix}': portfolio_out[portfolio_name],
        f'Ritorno{suffix}': portfolio_returns[portfolio_name],
        f'Volatilità{suffix}': portfolio_volatility_matrix[portfolio_name],
        f'Beta{suffix}': beta_mobile_df[portfolio_name]
    })

    # Taglia le righe iniziali (NaN)
    feature_matrix[portfolio_name] = feature_matrix[portfolio_name].iloc[W:]

    print(f"\nMatrice delle Feature per {portfolio_name}:")
    print(feature_matrix[portfolio_name].head())




Matrice delle Feature per Portafoglio 1:
    Prezzo_P1  Ritorno_P1  Volatilità_P1   Beta_P1
30  38.947945    0.001595       0.152615  0.751738
31  39.288996    0.005556       0.148774  0.765721
32  39.669954    0.001805       0.146090  0.757081
33  39.625630   -0.000687       0.143219  0.735515
34  39.767830    0.003617       0.139147  0.738387

Matrice delle Feature per Portafoglio 2:
     Prezzo_P2  Ritorno_P2  Volatilità_P2   Beta_P2
30  114.080890    0.000647       0.138493  0.715610
31  114.971844    0.003213       0.132046  0.710300
32  114.379005    0.003045       0.129023  0.697549
33  114.639824   -0.002668       0.123776  0.658279
34  114.018570    0.000129       0.121798  0.685960

Matrice delle Feature per Portafoglio 3:
    Prezzo_P3  Ritorno_P3  Volatilità_P3   Beta_P3
30  58.139538    0.001286       0.118917  0.644776
31  58.806683    0.006985       0.115925  0.652700
32  59.120827    0.000524       0.109163  0.624869
33  58.948026   -0.001977       0.103266  0.587435
3

In [16]:
#affianco tutte le matrici tenendo solo le righe comuni e avendo prezzo p1...betap1, prezzo p2...betap2, ...
final_feature_matrix = pd.DataFrame()
for p in range(num_portfolios):
    portfolio_name = f"Portafoglio {p+1}"
    if final_feature_matrix.empty:
        final_feature_matrix = feature_matrix[portfolio_name]
    else:
        final_feature_matrix = final_feature_matrix.join(feature_matrix[portfolio_name], how='inner')

        
print("\nMatrice Finale delle Feature per tutti i Portafogli:")
print(f"Dimensione (Giorni x Feature Totali): {final_feature_matrix.shape}")
print(final_feature_matrix.head())



Matrice Finale delle Feature per tutti i Portafogli:
Dimensione (Giorni x Feature Totali): (2486, 80)
    Prezzo_P1  Ritorno_P1  Volatilità_P1   Beta_P1   Prezzo_P2  Ritorno_P2  \
30  38.947945    0.001595       0.152615  0.751738  114.080890    0.000647   
31  39.288996    0.005556       0.148774  0.765721  114.971844    0.003213   
32  39.669954    0.001805       0.146090  0.757081  114.379005    0.003045   
33  39.625630   -0.000687       0.143219  0.735515  114.639824   -0.002668   
34  39.767830    0.003617       0.139147  0.738387  114.018570    0.000129   

    Volatilità_P2   Beta_P2  Prezzo_P3  Ritorno_P3  ...  Volatilità_P18  \
30       0.138493  0.715610  58.139538    0.001286  ...        0.154468   
31       0.132046  0.710300  58.806683    0.006985  ...        0.150687   
32       0.129023  0.697549  59.120827    0.000524  ...        0.145470   
33       0.123776  0.658279  58.948026   -0.001977  ...        0.141294   
34       0.121798  0.685960  58.607242   -0.001472  .